In [1]:
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition
import numpy as np

In [2]:
def print_rota_otima(Modelo):
    lista_produtores_ordenados = [(i, value(Modelo.y1[i])) for i, v in Modelo.y1.items() 
                                  if value(Modelo.y1[i]) != 0 or i == 0]
    lista_produtores_ordenados.sort(key=lambda x: x[1])

    print('Rota ótima dia 1:')
    print(' -> '.join(str(p) for p, _ in lista_produtores_ordenados))

    lista_produtores_ordenados = [(i, value(Modelo.y2[i])) for i, v in Modelo.y2.items() 
                                  if value(Modelo.y2[i]) and i != 0]
    lista_produtores_ordenados.sort(key=lambda x: x[1])

    print('Rota ótima dia 2:')
    print(' -> '.join(str(p) for p, _ in lista_produtores_ordenados))


# Planejamento de Coleta de Leite

---
### Descrição do Problema

Uma empresa de processamento de leite e derivados é responsável pela coleta de leite produzido por 20 fazendeiros, de forma que o leite seja entregue ao depósito, ou laticínio. A empresa possui um caminhão tanque com capacidade de 80.000 litros de leite. Dos 20 produtores/fazendeiros, 11 são de pequeno porte, de forma que sua produção pode ser coletada em dias alternados (ou seja, dia sim, dia não). Os 9 outros produtores precisam ter suas produções coletadas diariamente. As localizações dos produtores e do laticínio, bem como as demandas de coleta (por visita, ou seja, diária para os que devem ser visitados diariamente e total, referente a dois dias, para os demais) são apresentadas na Tabela 1. Note que a origem do sistema de coordenadas é colocada no ponto 1, que representa o laticínio.

Planeje a rota do caminhão tanque, de forma que o veículo percorra a mínima distância possível. Ou seja, deve-se planejar a rota para dois dias de coleta. Em cada um destes dias, os clientes de grande porte devem ser visitados. No primeiro dia, alguns dos clientes de pequeno porte podem ser visitados. Os que não forem visitados no primeiro dia devem ser visitados no segundo dia de planejamento. A capacidade do veículo deve ser respeitada em cada um dos dias. Assume-se que o veículo parte e retorna ao depósito em cada um destes dias.

[Mais detalhes do problema](exercicio-coleta-leite.pdf)


Este trabalho descreve a implementação do modelo MTZ (Miller-Tucker-Zemlin) e sua versão reforçada para resolver o Travelling Salesman Problem (TSP), um desafio clássico em otimização combinatória. Em uma instância específica, o TSP é contextualizado da seguinte forma:

* Uma empresa de laticínios busca otimizar a rota de seu caminhão-tanque, responsável pela coleta de leite em vários fornecedores, visando minimizar a distância total percorrida.
* O planejamento logístico deve cobrir dois dias consecutivos e considerar não apenas a distância entre os fazendeiros, mas também a frequência de visitas a cada cliente e a capacidade de carga do caminhão.

O objetivo principal é encontrar a rota mais eficiente para o deslocamento do veículo. Isso será alcançado através da formulação matemática do modelo MTZ para o problema de otimização combinatória mencionado, com o intuito de explorar e compreender as implicações resultantes das decisões de modelagem.

## Parâmetros para a modelagem

In [3]:
# Lista de produtores, representando os índices dos produtores
produtores = list(range(0, 21))

# Dicionário de demandas dos produtores, onde a chave é o índice do produtor 
# e o valor é a demanda correspondente
demandas = {0: 0, 1: 5000, 2: 4000, 3: 3000, 4: 6000, 5: 7000, 6: 3000, 7: 4000, 8: 6000, 
                    9: 5000, 10: 4000, 11: 7000, 12: 3000, 13: 4000, 14: 5000, 15: 6000, 16: 8000, 
                    17: 5000, 18: 7000, 19: 6000, 20: 6000}

# Capacidade total do veículo de coleta
capacidade = 80000

# Dicionário de frequência de coleta dos produtores, onde a chave é o índice do produtor 
# e o valor é a frequência de coleta ('d' para diária, 'alt' para alternada)
frequencia_coleta = {i: 'd' if i < 10 else 'alt' for i in range(21)}

# Dicionário de coordenadas dos produtores, onde a chave é o índice do produtor 
# e o valor é uma tupla representando as coordenadas (x, y) do produtor
coordenadas = {i: coord for i, coord in enumerate([(0, 0), (-30, 30), (10, 110), (40, 70), (-50, 90),
               (-50, -20), (-40, -70), (60, 0), (30, -60), (-10, -30),
               (0, -60), (60, 40), (20, 50), (-20, 80), (60, 100),
               (10, 80), (-30, 10), (-60, 50), (20, 90), (-60, -50), (50, -40)])}


In [4]:
def calcula_distancia_produtores(p1, p2):
        return  np.sqrt((coordenadas[p1][0] - coordenadas[p2][0])**2 + (coordenadas[p1][1] - 
                                                                            coordenadas[p2][1])**2)

## Variáveis de Decisão

In [5]:
Modelo = ConcreteModel()

# Define variáveis booleanas que indicam se a aresta X(i,j) existe, 
# ou seja, se o produtor i está conectado ao produtor j
Modelo.x1 = Var(produtores, produtores, within=Boolean)
Modelo.x2 = Var(produtores, produtores, within=Boolean)

# Define variáveis que representam o numero de produtores visitados 
# antes de visitar o produtor i nos dias 1 e 2 
Modelo.y1 = Var(produtores, within=NonNegativeIntegers)
Modelo.y2 = Var(produtores, within=NonNegativeIntegers)

# Cria uma lista com os indices dos produtores menores com base na frequência de coleta
produtores_alt = [p for p in produtores if frequencia_coleta[p] == 'alt']

# Define variáveis booleanas que indicam o dia de visita para os produtores menores 
# (0 para o primeiro dia, 1 para o segundo dia)
Modelo.z = Var(produtores_alt, within=Boolean)

## Definindo o objetivo 
### (Minimizar a distância percorrida nos dois dias)

In [6]:
# Cálculo da distância total percorrida no dia 1
distancia1 = sum(sum(Modelo.x1[i, j] * calcula_distancia_produtores(i, j) 
                     for j in produtores) for i in produtores)

# Cálculo da distância total percorrida no dia 2
distancia2 = sum(sum(Modelo.x2[i, j] * calcula_distancia_produtores(i, j) 
                     for j in produtores) for i in produtores)

# Função objetivo do modelo, define o objetivo de minimizar distância total
Modelo.obj = Objective(expr=(distancia1 + distancia2), sense=minimize)

## Definindo as restrições

In [7]:
Modelo.restricoes = ConstraintList()

# Garante 2 visitas aos produtores maiores 
# e 1 visita para alternados entre o par de dias de coleta
for i in produtores:
    Modelo.restricoes.add(Modelo.x1[i, i] == 0)
    Modelo.restricoes.add(Modelo.x2[i, i] == 0)
    if frequencia_coleta[i] == 'd':
        Modelo.restricoes.add(sum(Modelo.x1[i, j] for j in produtores if j != i) == 1)
        Modelo.restricoes.add(sum(Modelo.x2[i, j] for j in produtores if j != i) == 1)

    else:
        Modelo.restricoes.add(sum( Modelo.x1[i, j] for j in produtores if j != i) ==  Modelo.z[i])
        Modelo.restricoes.add(sum( Modelo.x2[i, j] for j in produtores if j != i) ==  1 - Modelo.z[i])
   
for j in produtores:
    if frequencia_coleta[j] == 'd':
        Modelo.restricoes.add(sum(Modelo.x1[i, j] for i in produtores if j != i) == 1)
        Modelo.restricoes.add(sum(Modelo.x2[i, j] for i in produtores if j != i) == 1)
    else:
        Modelo.restricoes.add(sum( Modelo.x1[i, j] for i in produtores if j != i) ==  Modelo.z[j])
        Modelo.restricoes.add(sum( Modelo.x2[i, j] for i in produtores if j != i) ==  1 - Modelo.z[j])

# Garante que a demanda coletada em cada dia não exceda a capacidade total do veículo
demanda_fixa = sum(demandas[p] for p in produtores if frequencia_coleta[p] == 'd')
Modelo.restricoes.add(demanda_fixa + sum(demandas[i] * Modelo.z[i] for i in produtores_alt ) <= capacidade)
Modelo.restricoes.add(demanda_fixa + sum(demandas[i] * (1-Modelo.z[i]) for i in produtores_alt) <= capacidade)

# Garante que a rota comece no ponto de partida (zero)
Modelo.restricoes.add(Modelo.y1[0] == 0)
Modelo.restricoes.add(Modelo.y2[0] == 0)
for i in produtores[1:]:
    Modelo.restricoes.add(Modelo.y1[i]>=0)
    Modelo.restricoes.add(Modelo.y2[i]>=0)

# Calcula o número máximo de vértices possíveis de serem visitados em uma única rota
# considerando sempre a visita de todos produtores grandes
soma = demanda_fixa
n = 10
# Filtrando as demandas apenas para os produtores menores
demandas_produtores_alt = {k: v for k, v in demandas.items() if k in produtores_alt}
# Ordenando as demandas dos produtores menores
for k, v in sorted(demandas_produtores_alt.items(), key=lambda x: x[1]):
    soma += v
    if soma > capacidade:
        break
    n += 1

## Formulação Fraca
As seguintes restrições garantem a conexão completa do grafo da solução, assegurando a existência de um único circuito ótimo que visite todos os vértices em um dado dia. No entanto, observe que essa desigualdade apresenta uma considerável folga para a maioria dos pares de vértices. Isso se reflete durante a etapa de relaxação linear, onde o limite dual alcançado fica significativamente distante do valor ótimo da função objetivo do problema de programação inteira. Como resultado, o algoritmo de Branch-and-Bound pode enfrentar dificuldades na convergência para encontrar a solução desejada, devido à modelagem fraca do problema.

In [8]:
"""
for i in produtores:
    for j in produtores[1:]:
        Modelo.restricoes.add(Modelo.y1[j] >= Modelo.y1[i] + Modelo.x1[i, j] * (n - 1) - (n - 2))
        Modelo.restricoes.add(Modelo.y2[j] >= Modelo.y2[i] + Modelo.x2[i, j] * (n - 1) - (n - 2))

"""

'\n\nfor i in produtores:\n    for j in produtores[1:]:\n        Modelo.restricoes.add(Modelo.y1[j] >= Modelo.y1[i] + Modelo.x1[i, j] * (n - 1) - (n - 2))\n        Modelo.restricoes.add(Modelo.y2[j] >= Modelo.y2[i] + Modelo.x2[i, j] * (n - 1) - (n - 2))\n\n'

## Formulação Fortalecida

O novo conjunto de restrições de desigualdade é mais restritiva em comparação com a anterior. Portanto, essa restrição é menos flexível e estabelece um limite inferior mais próximo do valor da função objetivo do problema de otimização combinatória, obtido através da relaxação linear. Como resultado, o algoritmo de Branch-and-Bound tende a se tornar mais eficiente, graças a uma formulação mais precisa.


In [9]:
for i in produtores:
    for j in produtores[1:]:
        Modelo.restricoes.add(Modelo.y1[j] >= Modelo.y1[i] + Modelo.x1[i, j] * (n - 1) 
                              + Modelo.x1[j, i] * (n - 3) - (n - 2))
        Modelo.restricoes.add(Modelo.y2[j] >= Modelo.y2[i] + Modelo.x2[i, j] * (n - 1) 
                              + Modelo.x2[j, i] * (n - 3) - (n - 2))

## Resolvendo:

In [12]:
# Resolver o modelo
solver = SolverFactory('glpk', solver_io='lp')
solver.options['tmlim'] = 120

results = solver.solve(Modelo, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --tmlim 120 --write C:\Users\CAU~1\AppData\Local\Temp\tmpz_2df21t.glpk.raw
 --wglp C:\Users\CAU~1\AppData\Local\Temp\tmppbqq5k29.glpk.glp --cpxlp C:\Users\CAU~1\AppData\Local\Temp\tmpqxrpre1h.pyomo.lp
Reading problem data from 'C:\Users\CAU~1\AppData\Local\Temp\tmpqxrpre1h.pyomo.lp'...
C:\Users\CAU~1\AppData\Local\Temp\tmpqxrpre1h.pyomo.lp:9928: warning: lower bound of variable 'x843' redefined
C:\Users\CAU~1\AppData\Local\Temp\tmpqxrpre1h.pyomo.lp:9928: warning: upper bound of variable 'x843' redefined
1010 rows, 935 columns, 5070 non-zeros
935 integer variables, 893 of which are binary
10821 lines were read
Writing problem data to 'C:\Users\CAU~1\AppData\Local\Temp\tmppbqq5k29.glpk.glp'...
8844 lines were written
GLPK Integer Optimizer, v4.65
1010 rows, 935 columns, 5070 non-zeros
935 integer variables, 893 of which are binary
Preprocessing...
40 constraint coefficient(s) were reduced
886 rows, 891 columns

## Resultados:

In [11]:
distancia_total = value(Modelo.obj)
print_rota_otima(Modelo)
# Mostrar resultados
print("\nDistância percorrida nos dois dias: {:.2f} Km".format(distancia_total))
print(f"\nCapacidade ociosa (1º dia): {capacidade - (demanda_fixa + sum(demandas[i] * (1-value(Modelo.z[i])) for i in produtores_alt))} L")
print(f"\nCapacidade ociosa (2º dia): {capacidade - (demanda_fixa + sum(demandas[i] * value(Modelo.z[i]) for i in produtores_alt ))} L")

Rota ótima dia 1:
0 -> 9 -> 5 -> 19 -> 6 -> 8 -> 7 -> 11 -> 3 -> 18 -> 2 -> 4 -> 17 -> 1 -> 16 -> 15
Rota ótima dia 2:
9 -> 11 -> 16 -> 18 -> 5 -> 6 -> 10 -> 8 -> 20 -> 7 -> 12 -> 3 -> 14 -> 2 -> 15 -> 13 -> 4 -> 1 -> 17 -> 19

Distância percorrida nos dois dias: 1230.66 Km

Capacidade ociosa (1º dia): 9000.0 L

Capacidade ociosa (2º dia): 4000.0 L
